In [1]:
# plotting magic
%matplotlib inline

# built-in libraries
from pathlib import Path

# 3rd party libraries I've installed
import cv2
import matplotlib.pyplot as plt
import numpy as np
import rawpy
from PIL import Image

# local libraries I've downloaded or created
import img_qc.exiftool as exiftool
import img_qc.img_qc as img_qc

In [2]:
data_directory_path = Path('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film')

In [3]:
data_directory_path

PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film')

In [4]:
# import and process negative using an LCC file for settings

# get list of all raw files in data directory
raw_image_paths_list = sorted(data_directory_path.glob('*.RAF'))  # *.RAF is the raw imaging file from our Fuji GFX 50s cameras

number_of_raw_images = len(raw_image_paths_list)
print(f'{number_of_raw_images} raw images')

17 raw images


In [5]:
raw_image_paths_list[:3]

[PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film/MS3892-B2-S11-F35_001.RAF'),
 PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film/MS3892-B2-S11-F35_002.RAF'),
 PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film/MS3892-B2-S11-F35_003.RAF')]

In [6]:
lcc_path = raw_image_paths_list[-2]
target_path = raw_image_paths_list[-1]
image_paths_list = [x for x in raw_image_paths_list if x != lcc_path and x != target_path]
image_paths_list[0]

PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film/MS3892-B2-S11-F35_001.RAF')

In [7]:
lcc_path, target_path

(PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film/bennett_pack-film_-001.RAF'),
 PosixPath('/Users/jmoor167/Documents/GitHub/utk_BennettPhotographCollection/data/bennett_pack-film/bennett_pack-film_-002.RAF'))

In [ ]:
# load image with context manager that closes it afterwards, but it also means we can't re-process the negative data

# process first image with default settings with rawpy
with rawpy.imread(str(image_paths_list[0])) as raw_negative:
    print('loaded_image')
    rgb_negative = raw_negative.postprocess()
plt.imshow(rgb_negative)

In [8]:
# load image without closing so we can do multiple raw conversions in the cells below
raw_negative = rawpy.imread(str(image_paths_list[0]))

In [ ]:
raw_negative.raw_colors_visible

In [ ]:
pil_image = Image.fromarray(raw_negative.raw_colors_visible)
pil_image.save(data_directory_path.joinpath('test.tif'))

In [ ]:
# how to close the opened file
raw_negative.close()

In [ ]:
plt.imshow(raw_negative.raw_image)

In [ ]:
plt.imshow(raw_negative.raw_image_visible)

In [ ]:
pil_image = Image.fromarray(raw_negative.raw_image_visible)
pil_image.save(data_directory_path.joinpath('test.tif'))

In [ ]:
!open {str(data_directory_path)}

In [ ]:
rawpy.flags

In [9]:
# time to get 8-bit linear, half_size, unprocessed image from RAW
time_8bit_halfsize = %timeit -o rgb_negative_linear = raw_negative.postprocess(half_size=True, output_color=rawpy.ColorSpace.raw, gamma=(1, 1), user_wb=[1.0, 1.0, 1.0, 1.0], no_auto_bright=True)

355 ms ± 6.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
round(time_8bit_halfsize.average, 2)

0.36

In [11]:
# get time to process 16-bit linear, half_size, unprocessed image from RAW
time_16bit_halfsize = %timeit -o rgb_negative_linear = raw_negative.postprocess(half_size=True, output_color=rawpy.ColorSpace.raw, gamma=(1, 1), user_wb=[1.0, 1.0, 1.0, 1.0], no_auto_bright=True, output_bps=16)

344 ms ± 3.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# get times for 8-bit, full_size images
time_8bit_fullsize = %timeit -o rgb_negative_linear = raw_negative.postprocess(half_size=False, output_color=rawpy.ColorSpace.raw, gamma=(1, 1), user_wb=[1.0, 1.0, 1.0, 1.0], no_auto_bright=True, output_bps=8)

4.89 s ± 49.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
# get times for 16-bit, full_size images
time_16bit_fullsize = %timeit -o rgb_negative_linear = raw_negative.postprocess(half_size=False, output_color=rawpy.ColorSpace.raw, gamma=(1, 1), user_wb=[1.0, 1.0, 1.0, 1.0], no_auto_bright=True, output_bps=16)

4.96 s ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
# bad, hacky method of getting variable names as string
variable_names = [time_8bit_halfsize, time_16bit_halfsize, time_8bit_fullsize, time_16bit_fullsize]

for variable in variable_names:
    name = [ k for k,v in locals().items() if v is variable][0]
    print(len(name), name)

18 time_8bit_halfsize
19 time_16bit_halfsize
18 time_8bit_fullsize
19 time_16bit_fullsize


In [15]:
for variable in variable_names:
    
    name = [ k for k,v in locals().items() if v is variable][0]
    
    print(f'Average for {name:20}', round(variable.average, 2))

Average for time_8bit_halfsize   0.36
Average for time_16bit_halfsize  0.34
Average for time_8bit_fullsize   4.89
Average for time_16bit_fullsize  4.96


In [ ]:
rawpy.libraw_version

In [ ]:
raw_negative.raw_pattern

In [16]:
# was going to try and demosaic the RAW file just using the green pixels, but this is something to figure out later
# RIGHT NOW: just demosaic and get use the green channel (BEFORE inversion!)

In [ ]:
# convert 16-bit to 8-bit values for displaying with matplotlib
rgb_negative_linear_8bit = (rgb_negative_linear/256).astype('uint8')

plt.imshow(rgb_negative_linear_8bit)

In [ ]:
# get flat-fielding frame with rawpy
image_path = Path('/Users/jeremy/Pictures/bennett_stitching_test/Capture/bennett_single-shot_003.RAF')
flatfield = rawpy.imread(str(image_path))
rgb_flatfield = flatfield.postprocess()

plt.imshow(rgb_flatfield)

In [ ]:
# get 16-bit, linear, unprocessed flat-fielding frame
rgb_flatfield_linear = flatfield.postprocess(output_color=rawpy.ColorSpace.raw, gamma=(1, 1), 
                                               user_wb=[1.0, 1.0, 1.0, 1.0], no_auto_bright=True, output_bps=16)

# convert 16-bit to 8-bit values for displaying with matplotlib
rgb_flatfield_linear_8bit = (rgb_flatfield_linear/256).astype('uint8')

plt.imshow(rgb_flatfield_linear_8bit)

In [ ]:
# 16-bit image is not on a 2**8 scale, but 2**16
2**16

In [ ]:
# get pixels with maximum value
np.where(rgb_flatfield_linear == 65535)

In [ ]:
# output is y, x, channel
rgb_flatfield_linear[6110, 4841, 1]

In [ ]:
250*256

In [ ]:
np.where(rgb_flatfield_linear > 64000)

In [ ]:
# how many pixels?
len(np.where(rgb_flatfield_linear >= 64000)[0])

In [ ]:
# if you want to write the images out to disk now
imageio.imwrite((data_directory_path.joinpath('negative_linear.tif')), rgb_negative_linear)
imageio.imwrite((data_directory_path.joinpath('flatfield_linear.tif')), rgb_flatfield_linear)
!open {str(data_directory_path)}

In [ ]:
# in photoshop i would 

In [ ]:
flatfielded = rgb_negative_linear / rgb_flatfield_linear

# convert 16-bit to 8-bit values for displaying with matplotlib
flatfielded_8bit = (flatfielded/256).astype('uint8')

plt.imshow(flatfielded_8bit)

Having issues because our flatfielding image has pixels with value 0

Divide by zero is bad!

In [ ]:
# get list of pixels where a value is 0
zero_list = np.where(rgb_flatfield_linear == 0)

# produces 3 arrays: y, x, channel
zero_list

In [ ]:
# loop through list of pixels with value zero and print out x, y, & color channel

print('x y color channel')
for index, y in enumerate(zero_list[0]):
    x = zero_list[1][index]
    channel = zero_list[2][index]
    if channel == 0:
        color = 'r'
    elif channel == 1:
        color = 'g'
    else:
        color = 'b'
    print(x, y, color)

In [ ]:
# need to blur LCC image before using as flatfield
# best practice might be to take the picture out of focus?
# take multiple pictures?

In [ ]:
# can only cv2.medianBlur 16-bit images with a kernel size of 5 or 8
rgb_flatfield_linear_blurred = cv2.medianBlur(rgb_flatfield_linear, 5)

# convert 16-bit to 8-bit values for displaying with matplotlib
rgb_flatfield_linear_blurred_8bit = (rgb_flatfield_linear_blurred/256).astype('uint8')

plt.imshow(rgb_flatfield_linear_blurred_8bit)

In [ ]:
# after blurring get list of pixels where a value is 0
zero_list = np.where(rgb_flatfield_linear_blurred == 0)

# produces 3 arrays: y, x, channel
zero_list

In [ ]:
# after blurring get list of pixels where a value is 65335
zero_list = np.where(rgb_flatfield_linear_blurred == 65335)

# produces 3 arrays: y, x, channel
zero_list

In [ ]:
rgb_flatfield_linear_blurred.max()

In [ ]:
63069//256

In [ ]:
# convert 16-bit to 8-bit values for higher medianBlur test
rgb_flatfield_linear_8bit = (rgb_flatfield_linear/256).astype('uint8')

# can only cv2.medianBlur 16-bit images with a kernel size of 5 or 8
rgb_flatfield_linear_blurred_8bit_test = cv2.medianBlur(rgb_flatfield_linear_8bit, 15)

plt.imshow(rgb_flatfield_linear_blurred_8bit_test)

In [ ]:
# get list of pixels where a value is 0
zero_list = np.where(rgb_flatfield_linear_blurred_8bit_test == 0)

# produces 3 arrays: y, x, channel
zero_list

In [ ]:
# get list of pixels where a value is 0
zero_list = np.where(rgb_flatfield_linear_blurred_8bit_test == 255)

# produces 3 arrays: y, x, channel
zero_list

In [ ]:
image_copy = rgb_flatfield_linear_blurred_8bit_test.copy()

black_pixels_mask = np.all(rgb_flatfield_linear_blurred_8bit_test == [0, 0, 0], axis=-1)

non_black_pixels_mask = np.any(rgb_flatfield_linear_blurred_8bit_test != [0, 0, 0], axis=-1)  
# or non_black_pixels_mask = ~black_pixels_mask

image_copy[black_pixels_mask] = [255, 255, 255]
image_copy[non_black_pixels_mask] = [0, 0, 0]

plt.imshow(image_copy)
plt.show()